THêm thư viện


In [35]:
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import pandas as pd
import matplotlib.pyplot as plt
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cityiscoding\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Trực quan hóa dữ liệu

In [36]:
file_path = 'Data\data.csv'  
df = pd.read_csv(file_path)
# Xóa cột 'flags' và 'category'
df = df.drop(['flags', 'category'], axis=1)
 #Xóa những ký tự trong ngoặc của cột ''
# Xóa ký tự trong ngoặc của cột 'patterns' và 'responses'
df['patterns'] = df['patterns'].apply(lambda x: re.sub(r'\{\{.*?\}\}', '', x))
df['responses'] = df['responses'].apply(lambda x: re.sub(r'\{\{.*?\}\}', '', x))
df.head(100)

,patterns,tag,responses
0,question about cancelling order,cancel_order,I've understood you have a question regarding ...
1,i have a question about cancelling oorder,cancel_order,I've been informed that you have a question ab...
2,i need help cancelling puchase,cancel_order,I can sense that you're seeking assistance wit...
3,I need to cancel purchase,cancel_order,I understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase",cancel_order,I'm sensitive to the fact that you're facing f...
...,...,...,...
95,I want help cancellkng purchase,cancel_order,I appreciate that you're seeking assistance wi...
96,I need help with cancelling order,cancel_order,I can relate to the understanding that you're ...
97,I want assistance to cancel order,cancel_order,I've been there that you're seeking assistance...
98,I want assistance to cancel order,cancel_order,I can see that you're seeking assistance to ca...


In [37]:
# Tạo một dictionary chứa thông tin intents
intents_data = {"intents": []}

# Duyệt qua từng dòng trong DataFrame
for index, row in df.iterrows():
    intent = {
        "tag": row['tag'],
        "patterns": [row['patterns']],
        "responses": [row['responses']]
    }
    # Thêm intent vào danh sách intents
    intents_data["intents"].append(intent)

# Chuyển dictionary thành JSON và lưu vào file với encoding utf-8
with open('Data/intents.json', 'w', encoding='utf-8') as json_file:
    json.dump(intents_data, json_file, indent=2, ensure_ascii=False)

print("Chuyển đổi thành công!")
df.head(35385)

Chuyển đổi thành công!


,patterns,tag,responses
0,question about cancelling order,cancel_order,I've understood you have a question regarding ...
1,i have a question about cancelling oorder,cancel_order,I've been informed that you have a question ab...
2,i need help cancelling puchase,cancel_order,I can sense that you're seeking assistance wit...
3,I need to cancel purchase,cancel_order,I understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase",cancel_order,I'm sensitive to the fact that you're facing f...
...,...,...,...
26867,I am waiting for a rebate of dollars,track_refund,Thank you for sharing your situation regarding...
26868,how to see if there is anything wrong with my ...,track_refund,Ensuring the accuracy of your restitution is o...
26869,I'm waiting for a reimbjrsement of,track_refund,"Firstly, I genuinely understand the importance..."
26870,I don't know what to do to see my reimbursemen...,track_refund,I've understood you're unsure about how to che...


## Tải dữ liệu file intents.json đã xữ lý

In [38]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

with open('Data\intents.json', encoding='utf-8') as file:
    data_file = file.read()
    intents = json.loads(data_file)

intents = json.loads(data_file)
import nltk
nltk.download('wordnet')
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()




[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cityiscoding\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Cityiscoding\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Pre-process data

Lặp lại các mẫu và mã hóa câu bằng cách sử dụng hàm nltk.word_tokenize() và nối từng từ vào danh sách từ. Tạo danh sách các lớp cho thẻ của mình.

In [39]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Bổ nghĩa từng từ và loại bỏ các từ trùng lặp khỏi danh sách

In [40]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

26872 documents
27 classes ['cancel_order', 'change_order', 'change_shipping_address', 'check_cancellation_fee', 'check_invoice', 'check_payment_methods', 'check_refund_policy', 'complaint', 'contact_customer_service', 'contact_human_agent', 'create_account', 'delete_account', 'delivery_options', 'delivery_period', 'edit_account', 'get_invoice', 'get_refund', 'newsletter_subscription', 'payment_issue', 'place_order', 'recover_password', 'registration_problems', 'review', 'set_up_shipping_address', 'switch_account', 'track_order', 'track_refund']
2821 unique lemmatized words ['#', '$', "'d", "'m", ',', '.', '00004587345current', '00108', '00123842current', '10', '113542617735902current', '1200dollars', '12588', '1499dollars', '1499for', '160dollars', '299dollars', '350dollars', '370795561790current', '370795561790will', '37777', '4', '6', '7', '732201349959current', '8', '85632', '9', 'a', 'aa', 'aabout', 'aaccount', 'aailable', 'aasistance', 'aat', 'abailable', 'abot', 'abou', 'aboujt'

## Huấn luyện và kiểm tra


In [41]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create a base word, in an attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in the current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features
random.shuffle(training)

# separate train_x and train_y
train_x = np.array([i[0] for i in training])
train_y = np.array([i[1] for i in training])

print("Huấn luyện thành công")


Huấn luyện thành công


## Tạo mô hình chatbox

In [42]:
from tensorflow.keras.optimizers import SGD

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons, and 3rd output layer contains the number of neurons
# equal to the number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=10, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("tạo mô hình thành cống")


Epoch 1/10


5375/5375 [==============================] - 19s 3ms/step - loss: 0.7737 - accuracy: 0.7568
Epoch 2/10
5375/5375 [==============================] - 17s 3ms/step - loss: 0.2816 - accuracy: 0.9121
Epoch 3/10
5375/5375 [==============================] - 18s 3ms/step - loss: 0.2415 - accuracy: 0.9270
Epoch 4/10
5375/5375 [==============================] - 17s 3ms/step - loss: 0.2141 - accuracy: 0.9390
Epoch 5/10
5375/5375 [==============================] - 17s 3ms/step - loss: 0.2257 - accuracy: 0.9374
Epoch 6/10
5375/5375 [==============================] - 18s 3ms/step - loss: 0.2255 - accuracy: 0.9409
Epoch 7/10
5375/5375 [==============================] - 18s 3ms/step - loss: 0.2274 - accuracy: 0.9408
Epoch 8/10
5375/5375 [==============================] - 17s 3ms/step - loss: 0.2329 - accuracy: 0.9393
Epoch 9/10
5375/5375 [==============================] - 17s 3ms/step - loss: 0.2784 - accuracy: 0.9338
Epoch 10/10
5375/5375 [==============================] - 16s 3ms/step -

c:\Users\Cityiscoding\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
from keras.models import load_model
import h5py

model_path = 'chatbot_model.h5'
with h5py.File(model_path, 'r') as file:
    model = load_model(file, compile=False)

import json
import random
intents = json.loads(open('Data\intents.json', encoding='utf-8-sig').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [ ]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## Giao diện người dùng

In [ ]:
import tkinter
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#c5f0e3"
TEXT_COLOR = "#000000"

# BG_GRAY = "#ABB2B9"
# BG_COLOR = "#1c172a"
# TEXT_COLOR = "#ffffff"


FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"


def send(event):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "Bạn: " + msg + '\n\n')
        ChatLog.config(foreground="#000000", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Nhân viên hổ trợ: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("Chatbot hô trợ")
base.resizable(width=FALSE, height=FALSE)
base.configure(width=800, height=800, bg=BG_COLOR)


#Create Chat window
ChatLog = Text(base, bd=0, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT_BOLD)
ChatLog.config(state=DISABLED)

head_label = Label(base, bg=BG_COLOR, fg=TEXT_COLOR, text="Chào mừng bạn đã đến với mục hổ trợ của PallMall.Shop", font=FONT_BOLD, pady=10)
head_label.place(relwidth=1)

line = Label(base, width=450, bg=BG_GRAY)


#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
ChatLog.focus()

#Create Button to send message
SendButton = Button(base, font=("Verdana", 12, 'bold'), text="GỬi", width="12", height=15,
                    bd=0, bg="#ed9061", activebackground="#3c9d9b", fg='#ffffff',
                    command=lambda: send(None))


#Create the box to enter message
EntryBox = Text(base, bg="white",width="29", height="5", font="Arial", background="#dddddd")
EntryBox.focus()
EntryBox.bind("<Return>", send)
#EntryBox.bind("<Return>", send)

# bottom label
# bottom_label = Label(base, bg=BG_GRAY, height=80)
# bottom_label.place(relwidth=1, rely=0.825)
        
# message entry box
#EntryBox = Entry(bottom_label, fg=TEXT_COLOR, font=FONT)
# msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, |relx=0.011)
# msg_entry.focus()


#Place all components on the screen
# scrollbar.place(relheight=1, relx=0.974)
# ChatLog.place(relheight=1, width=1)
# EntryBox.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
# SendButton.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.20)\

scrollbar.place(x=775,y=6, height=800)
line.place(x=0,y=35, height=1, width=770)
ChatLog.place(x=5,y=40, height=700, width=770)
EntryBox.place(x=0, y=740, height=60, width=600)
SendButton.place(x=600, y=740, height=60, width=175)
base.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Cityiscoding\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Cityiscoding\AppData\Local\Temp\ipykernel_8124\3147676889.py", line 25, in send
    res = chatbot_response(msg)
  File "C:\Users\Cityiscoding\AppData\Local\Temp\ipykernel_8124\287271095.py", line 2, in chatbot_response
    ints = predict_class(text, model)
  File "C:\Users\Cityiscoding\AppData\Local\Temp\ipykernel_8124\3368871726.py", line 4, in predict_class
    res = model.predict(np.array([p]))[0]
  File "c:\Users\Cityiscoding\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\CITYIS~1\AppData\Local\Temp\__autograph_generated_fileyrcjs5vp.py", line 15, in tf__predict_function
    retval_ = ag__.converted_call(ag__.ld(step_fun